In [5]:
from Helper_Functions import *
from Models import *
from Penalties import *
from docplex.mp.model import Model

In [6]:
# Test parameters
num_nodes_list = [5, 10, 15]  # Adjust as needed for larger tests
k_values = [2, 3, 4, 5]       # Values of K to test
edge_probability = 0.5        # Probability of edge creation
weighted = True
weight_range = 10             # Range for edge weights
num_graphs_per_combination = 5  # Number of graphs to test per combination

In [7]:
# Collect discrepancies
discrepancies = []

for num_nodes in num_nodes_list:
    for K in k_values:
        for graph_index in range(num_graphs_per_combination):
            # Generate a random graph
            G = generate_graph(num_nodes, edge_probability, weighted, weight_range)
            
            # Compute the naive penalties
            PENALTY_QUBO = naive_qubo_penalty(G, K)
            PENALTY_RQUBO = naive_rqubo_penalty(G, K)
            
            # Build models
            model_bqo = docplex_BQO(G, K, name=f"BQO_n{num_nodes}_k{K}")
            model_qubo = docplex_QUBO(G, K, PENALTY_QUBO, name=f"QUBO_n{num_nodes}_k{K}")
            model_rbqo = docplex_RBQO(G, K, name=f"RBQO_n{num_nodes}_k{K}")
            model_rqubo = docplex_RQUBO(G, K, PENALTY_RQUBO, name=f"RQUBO_n{num_nodes}_k{K}")
            
            # Solve models and get variable assignments
            solution_bqo = get_solution_values(model_bqo)
            solution_qubo = get_solution_values(model_qubo)
            solution_rbqo = get_solution_values(model_rbqo)
            solution_rqubo = get_solution_values(model_rqubo)
            
            # Check if any model failed to find a solution
            if None in [solution_bqo, solution_qubo, solution_rbqo, solution_rqubo]:
                print(f"Solver failed for num_nodes={num_nodes}, K={K}, graph_index={graph_index}")
                continue
            
            # Compute BQO objective values for each solution
            obj_bqo = compute_bqo_objective(solution_bqo, G, K)
            obj_qubo = compute_bqo_objective(solution_qubo, G, K)
            
            # For RBQO and RQUBO, reconstruct the full assignments
            full_assignment_rbqo = rbqo_to_bqo(solution_rbqo, G, K)
            obj_rbqo = compute_bqo_objective(full_assignment_rbqo, G, K)

            full_assignment_rqubo = rqubo_to_bqo(solution_rqubo, G, K)
            obj_rqubo = compute_bqo_objective(full_assignment_rqubo, G, K)
            
            # Compare objective values
            objs = [obj_bqo, obj_qubo, obj_rbqo, obj_rqubo]
            max_obj = max(objs)
            min_obj = min(objs)
            if not np.isclose(max_obj, min_obj, atol=1e-6):
                discrepancies.append({
                    'num_nodes': num_nodes,
                    'K': K,
                    'graph_index': graph_index,
                    'obj_bqo': obj_bqo,
                    'obj_qubo': obj_qubo,
                    'obj_rbqo': obj_rbqo,
                    'obj_rqubo': obj_rqubo
                })
                print(f"Discrepancy found for num_nodes={num_nodes}, K={K}, graph_index={graph_index}")

In [8]:
print("\nDiscrepancies Summary:")
for discrepancy in discrepancies:
    print(f"num_nodes={discrepancy['num_nodes']}, K={discrepancy['K']}, graph_index={discrepancy['graph_index']}")
    print(f"  BQO Objective:   {discrepancy['obj_bqo']}")
    print(f"  QUBO Objective:  {discrepancy['obj_qubo']}")
    print(f"  RBQO Objective:  {discrepancy['obj_rbqo']}")
    print(f"  RQUBO Objective: {discrepancy['obj_rqubo']}\n")


Discrepancies Summary:
